In [1]:
%run ../common.ipynb

In [2]:
pip install faker

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 4.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.8/346.8 kB 2.4 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyspark.sql import SparkSession
from faker import Faker
import random

# Initialize Spark Session with MySQL JDBC Driver
spark = SparkSession.builder.appName("load_operation_data") \
    .config("spark.jars.packages", "mysql:mysql-connector-java:8.0.33") \
    .getOrCreate()


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


:: loading settings :: url = jar:file:/home/glue_user/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/glue_user/.ivy2/cache
The jars for the packages stored in: /home/glue_user/.ivy2/jars
mysql#mysql-connector-java added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1136d83c-cabb-4bee-affc-5e62ae5f6c99;1.0
	confs: [default]
mysql#mysql-connector-java;8.0.33 is relocated to com.mysql#mysql-connector-j;8.0.33. Please update your dependencies.
	found mysql#mysql-connector-java;8.0.33 in central
	found com.mysql#mysql-connector-j;8.0.33 in central
	found com.google.protobuf#protobuf-java;3.21.9 in central
downloading https://repo1.maven.org/maven2/com/mysql/mysql-connector-j/8.0.33/mysql-connector-j-8.0.33.jar ...
	[SUCCESSFUL ] com.mysql#mysql-connector-j;8.0.33!mysql-connector-j.jar (2397ms)
downloading https://repo1.maven.org/maven2/com/google/protobuf/protobuf-java/3.21.9/protobuf-java-3.21.9.jar ...
	[SUCCESSFUL ] com.google.protobuf#protobuf-java;3.21.9!protobuf-java.jar(bundle) (1336ms)
:: resolution report :: resol

In [4]:
# Initialize Faker for realistic data
fake = Faker()

# Generate Customers
def generate_customers(n):
    return spark.createDataFrame(
        [(i, fake.first_name(), fake.last_name(), fake.email(), fake.phone_number(),
          fake.address(), fake.city(), fake.state(), fake.zipcode(), fake.country()) 
         for i in range(1, n+1)], 
        ["customer_id", "first_name", "last_name", "email", "phone", "address", "city", "state", "zip_code", "country"]
    )

In [5]:
# Generate Data
customers_df = generate_customers(1000)
customers_df.show()

+-----------+-----------+---------+--------------------+--------------------+--------------------+-------------------+-------------+--------+----------------+
|customer_id| first_name|last_name|               email|               phone|             address|               city|        state|zip_code|         country|
+-----------+-----------+---------+--------------------+--------------------+--------------------+-------------------+-------------+--------+----------------+
|          1|    Michael|  Rodgers|katiecasey@exampl...|+1-664-301-3316x9...|USS Dudley\nFPO A...|        North Traci|        Idaho|   38917|     Saint Lucia|
|          2|      Paula|    Berry|carolyn15@example...|  971-396-8026x98919|PSC 9301, Box 951...|       Hornechester|     Michigan|   14169|          Turkey|
|          3|     Angela| Lawrence|  kchung@example.org|   616-578-2644x4229|884 Brandi Pike A...|         New Kelsey|     Maryland|   38467|         Lesotho|
|          4|      Nancy|      May| carol17@ex

In [6]:
# Remove duplicate emails before writing to MySQL
customers_df = customers_df.dropDuplicates(["email"])

customers_df.write.jdbc(url=USER_MYSQL_URL, table="Customers", mode="append", properties=MYSQL_PROPERTIES)

print("✅ Data successfully written to MySQL!")

✅ Data successfully written to MySQL!


In [7]:





# Generate Products
def generate_products(n):
    categories = ["Electronics", "Clothing", "Home & Kitchen", "Books", "Toys", "Sports"]
    return spark.createDataFrame(
        [(i, fake.word().capitalize(), fake.sentence(nb_words=6), 
          random.choice(categories), round(random.uniform(5, 500), 2), random.randint(10, 1000)) 
         for i in range(1, n+1)], 
        ["product_id", "name", "description", "category", "price", "stock_quantity"]
    )




# Generate Payments
def generate_payments(n, max_order_id):
    return spark.createDataFrame(
        [(i, random.randint(1, max_order_id), fake.date_this_decade(), 
          random.choice(["Credit Card", "PayPal", "Gift Card", "Bank Transfer"]), 
          round(random.uniform(10, 500), 2), random.choice(["Success", "Failed", "Pending"])) 
         for i in range(1, n+1)], 
        ["payment_id", "order_id", "payment_date", "payment_method", "amount", "status"]
    )

# Generate Purchase History
def generate_purchase_history(n, max_customer_id, max_order_id):
    return spark.createDataFrame(
        [(i, random.randint(1, max_customer_id), random.randint(1, max_order_id), 
          fake.date_this_decade(), round(random.uniform(10, 500), 2)) 
         for i in range(1, n+1)], 
        ["history_id", "customer_id", "order_id", "purchase_date", "amount_spent"]
    )

# Generate Usage History
def generate_usage_history(n, max_customer_id, max_product_id):
    return spark.createDataFrame(
        [(i, random.randint(1, max_customer_id), random.randint(1, max_product_id), 
          fake.date_this_decade(), fake.sentence(nb_words=10), 
          random.choice(["Returned", "Not Returned"])) 
         for i in range(1, n+1)], 
        ["usage_id", "customer_id", "product_id", "usage_date", "feedback", "return_status"]
    )

# Generate Transactions
def generate_transactions(n, max_customer_id, max_order_id, max_payment_id):
    return spark.createDataFrame(
        [(i, random.randint(1, max_customer_id), random.randint(1, max_order_id), 
          random.randint(1, max_payment_id), fake.date_this_decade(), 
          round(random.uniform(10, 500), 2), random.choice(["Purchase", "Refund"]), 
          random.choice(["Completed", "Failed", "Pending"])) 
         for i in range(1, n+1)], 
        ["transaction_id", "customer_id", "order_id", "payment_id", "transaction_date", 
         "amount", "transaction_type", "status"]
    )




In [8]:
products_df = generate_products(1000)

products_df.write.jdbc(url=PRODUCT_MYSQL_URL, table="Products", mode="append", properties=MYSQL_PROPERTIES)


In [9]:

customers_df = spark.read.jdbc(url=USER_MYSQL_URL, table="Customers", properties=MYSQL_PROPERTIES)

valid_customer_ids = [row["customer_id"] for row in customers_df.select("customer_id").collect()]

# ✅ Generate Orders using only valid customer IDs
def generate_orders(n):
    return spark.createDataFrame(
        [(i, random.choice(valid_customer_ids), fake.date_this_decade(), 
          random.choice(["Pending", "Shipped", "Delivered", "Cancelled"]), 
          round(random.uniform(10, 500), 2)) 
         for i in range(1, n+1)], 
        ["order_id", "customer_id", "order_date", "status", "total_amount"]
    )

orders_df = generate_orders(100000)
orders_df.show()

+--------+-----------+----------+---------+------------+
|order_id|customer_id|order_date|   status|total_amount|
+--------+-----------+----------+---------+------------+
|       1|         68|2022-07-22|  Shipped|      236.06|
|       2|        458|2021-09-13|Cancelled|       191.2|
|       3|        158|2021-03-12|  Shipped|      227.43|
|       4|        199|2020-10-18|Cancelled|      357.54|
|       5|        989|2021-06-28|  Pending|       72.94|
|       6|        418|2021-01-28|  Shipped|      108.65|
|       7|        379|2021-03-18|Delivered|      247.31|
|       8|        727|2020-08-13|  Pending|       59.05|
|       9|        648|2020-05-09|Delivered|      319.87|
|      10|        112|2024-07-11|  Pending|      183.45|
|      11|        455|2023-05-14|  Shipped|       24.82|
|      12|        123|2024-07-12|  Shipped|      309.55|
|      13|        655|2024-08-22|  Shipped|      396.34|
|      14|        938|2020-06-23|Delivered|      492.61|
|      15|        452|2023-04-0

In [10]:

orders_df.write.jdbc(url=ORDER_MYSQL_URL, table="Orders", mode="append", properties=MYSQL_PROPERTIES)

In [11]:

#orders_df.write.jdbc(url=MYSQL_URL, table="Orders", mode="append", properties=MYSQL_PROPERTIES)

orders_df = spark.read.jdbc(url=ORDER_MYSQL_URL, table="Orders", properties=MYSQL_PROPERTIES)
products_df = spark.read.jdbc(url=PRODUCT_MYSQL_URL, table="Products", properties=MYSQL_PROPERTIES)
valid_order_ids = [row["order_id"] for row in orders_df.select("order_id").collect()]
valid_product_ids = [row["product_id"] for row in products_df.select("product_id").collect()]
valid_customer_ids = [row["customer_id"] for row in customers_df.select("customer_id").collect()]
# ✅ Generate Order Items using only valid order_id & product_id
def generate_order_items(n):
    return spark.createDataFrame(
        [(i, random.choice(valid_order_ids), random.choice(valid_product_ids), 
          random.randint(1, 5), round(random.uniform(5, 500), 2)) 
         for i in range(1, n+1)], 
        ["order_item_id", "order_id", "product_id", "quantity", "unit_price"]
    )
order_items_df = generate_order_items(100000)
order_items_df.write.jdbc(url=ORDER_MYSQL_URL, table="Order_Items", mode="append", properties=MYSQL_PROPERTIES)


In [12]:
# Generate Payments
def generate_payments(n):
    return spark.createDataFrame(
        [(i,random.choice(valid_order_ids), fake.date_this_decade(), 
          random.choice(["Credit Card", "PayPal", "Gift Card", "Bank Transfer"]), 
          round(random.uniform(10, 500), 2), random.choice(["Success", "Failed", "Pending"])) 
         for i in range(1, n+1)], 
        ["payment_id", "order_id", "payment_date", "payment_method", "amount", "status"]
    )
payments_df = generate_payments(10000)
valid_payments_ids = [row["payment_id"] for row in payments_df.select("payment_id").collect()]

# Generate Purchase History
def generate_purchase_history(n):
    return spark.createDataFrame(
        [(i,random.choice(valid_customer_ids), random.choice(valid_order_ids), 
          fake.date_this_decade(), round(random.uniform(10, 500), 2)) 
         for i in range(1, n+1)], 
        ["history_id", "customer_id", "order_id", "purchase_date", "amount_spent"]
    )

payments_df.write.jdbc(url=ORDER_MYSQL_URL, table="Payments", mode="append", properties=MYSQL_PROPERTIES)
purchase_history_df = generate_purchase_history(1000)
purchase_history_df.distinct().write.jdbc(url=ORDER_MYSQL_URL, table="Purchase_History", mode="append", properties=MYSQL_PROPERTIES)


In [13]:

# Generate Usage History
def generate_usage_history(n):
    return spark.createDataFrame(
        [(i,random.choice(valid_customer_ids),random.choice(valid_product_ids),fake.date_this_decade(), fake.sentence(nb_words=10), 
          random.choice(["Returned", "Not Returned"])) 
         for i in range(1, n+1)], 
        ["usage_id", "customer_id", "product_id", "usage_date", "feedback", "return_status"]
    )

# Generate Transactions
def generate_transactions(n,):
    return spark.createDataFrame(
        [(i, random.choice(valid_customer_ids), random.choice(valid_order_ids), 
          random.choice(valid_payments_ids), fake.date_this_decade(), 
          round(random.uniform(10, 500), 2), random.choice(["Purchase", "Refund"]), 
          random.choice(["Completed", "Failed", "Pending"])) 
         for i in range(1, n+1)], 
        ["transaction_id", "customer_id", "order_id", "payment_id", "transaction_date", 
         "amount", "transaction_type", "status"]
    )
usage_history_df = generate_usage_history(1000)
usage_history_df.distinct().write.jdbc(url=ORDER_MYSQL_URL, table="Usage_History", mode="append", properties=MYSQL_PROPERTIES)

transactions_df = generate_transactions(1000)
transactions_df.distinct().write.jdbc(url=ORDER_MYSQL_URL, table="Transactions", mode="append", properties=MYSQL_PROPERTIES)

print("✅ Data successfully written to MySQL!")

✅ Data successfully written to MySQL!


In [14]:
from pyspark.sql.functions import when,rand,lit
# Generate random login data
def generate_logins(customers_df, num_attempts=50):
    logins = []
    for row in customers_df.collect():
        customer_id = row["customer_id"]
        num_logins = random.randint(1, num_attempts)  # Random login attempts per customer
        
        for _ in range(num_logins):
            ip = f"192.168.{random.randint(0, 255)}.{random.randint(0, 255)}"
            login_attempt = random.choice(["SUCCESS", "FAILURE"])
            login_date = f"2024-02-{random.randint(1, 28)}"
            
            logins.append((customer_id, ip, login_attempt, login_date))
    
    return logins

# Generate synthetic login data
login_data = generate_logins(customers_df)

# Create DataFrame
columns = ["customer_id", "ip_address", "login_attempt", "login_date"]
login_df = spark.createDataFrame(login_data, columns)

# Introduce suspicious activity (customers with >3 IPs or >10 attempts)
login_df = login_df.withColumn("is_suspicious", when((rand() < 0.2), lit(1)).otherwise(lit(0)))

# Write to MySQL (optional)
login_df.write.jdbc(url=USER_MYSQL_URL, table="LoginHistory", mode="append", properties=MYSQL_PROPERTIES)

# Show result
login_df.show()

+-----------+---------------+-------------+----------+-------------+
|customer_id|     ip_address|login_attempt|login_date|is_suspicious|
+-----------+---------------+-------------+----------+-------------+
|          1|  192.168.39.36|      FAILURE| 2024-02-8|            1|
|          1| 192.168.85.132|      FAILURE|2024-02-11|            0|
|          1|    192.168.3.0|      FAILURE| 2024-02-6|            1|
|          1|192.168.224.197|      FAILURE| 2024-02-2|            0|
|          1|  192.168.54.36|      SUCCESS|2024-02-10|            0|
|          1|192.168.123.248|      FAILURE|2024-02-24|            0|
|          1|192.168.104.243|      SUCCESS|2024-02-14|            0|
|          1| 192.168.126.81|      SUCCESS|2024-02-13|            0|
|          1|192.168.118.229|      FAILURE|2024-02-28|            0|
|          1|  192.168.95.87|      FAILURE|2024-02-23|            1|
|          1| 192.168.254.32|      SUCCESS|2024-02-22|            0|
|          1| 192.168.95.168|     